In [1]:
import sys
import logging

import numpy as np
import scipy as sp
import sklearn


%load_ext autoreload
%autoreload 2

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import seaborn as sns
sns.set_context("poster")
sns.set(rc={'figure.figsize': (16, 9.)})
sns.set_style("whitegrid")

import pandas as pd
pd.set_option("display.max_rows", 120)
pd.set_option("display.max_columns", 120)

from fox_toolbox.utils import xml_parser, rates, volatility

In [5]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm

S0 = 100.
r = 0.01
q =0.00
mu = r - q

ntime = 20
mc = 50

vol = 0.1
T = 100.
K = 100.

dt = T/ntime

def payoff(traj, K):
    for final in traj:
        yield (final - K) if final>K else 0
        
        
def Black(S0, r, q, T, K, vol):
    d1 = (np.log(S0/K) + (r - q + vol**2/2)*T)/(vol*np.sqrt(T))
    d2 = d1 - vol*np.sqrt(T)
    price = S0*np.exp(-q*T) * norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)
    return price

def ST(S0, r, q, T, K, vol, x):
    return S0*np.exp((r-q-vol**2/2)*T + vol*np.sqrt(T)*x)

noise = np.random.normal(0, 1., (ntime, mc))
traj = np.zeros([ntime+1, mc]) + S0

for t, rnd in zip(range(ntime), noise):
    traj[t+1] = traj[t]*(1 + mu*dt + vol*np.sqrt(dt)*rnd)

payoffs = list(payoff(traj[-1], K))
num_traj = np.mean(payoffs)/np.exp(r*T)

fwd = S0*np.exp(r*T)
v =  vol*np.sqrt(T)
black_price = volatility.BSPrice(fwd, K, v) * np.exp(-r*T)

STs = [ST(S0, r, q, T, K, vol, x) for x in np.random.normal(0, 1., mc)]
POs = [(x-K) if x>K else 0 for x in STs]
num_final = np.mean(POs)/np.exp(r*T)
print(num_traj, num_final, black_price)

54.32454241921172 66.22030729876795 67.88179748866278


In [6]:
from fox_toolbox.utils import volatility

In [9]:
from numpy import exp, sqrt
tvar = vol*sqrt(T)
F = S0 * exp(r*T)
volatility.BSPrice(F, K, tvar)*exp(-r*T)

67.88179748866278